# 4.2 - Procesos paralelos


![parallel](images/parallel.png)

$$$$

### Multiprocessing

Veamos en primer lugar [multiprocessing](https://docs.python.org/es/3.9/library/multiprocessing.html). Es una librería de Python que nos permite manejar hilos y procesos. La diferencia entre hilo y proceso es que un hilo ocurre dentro del espacio de memoria de un programa y un proceso es una copia completa del programa, por esta razón, los hilos son rápidos de crear y destruir además de que consumen poca memoria y los procesos son lentos de crear y destruir además de que requieren clonar el espacio de memoria del programa en otro lugar de la RAM, y esto es lento. Ejemplos de esto serían, subrutinas que recogen mensajes de un puerto de comunicaciones y los usan para actuar sobre emails almacenados en un servidor, desde el punto de vista del servidor, el cliente de correo sólo necesita usar el servidor durante un corto plazo de tiempo, porque envía un mensaje al servidor donde le indica lo que el usuario desea hacer, saber si hay mensajes nuevos, borrar un correo, moverlo... El servidor abre un hilo para atender a ese usuario y el hilo sólo vive mientras dure la conexión del usuario, una vez el usuario ha terminado el cliente de correo desconecta hasta nueva acción. Este proceso que he descrito es rápido, ocurre en milisegundos y generalmente se resuelve con hilos porque es más ligero para el sistema operativo y su vida media es especialmente corta, además de que el sistema podrá aceptar ciento o miles de conexiones por segundo y será ligero, rápido y eficiente en esta tarea.

La tendencia actual entre los desarrolladores es hacer una aplicaciones que sean rápidas en un sólo hilo y luego escalar a tantas instancias como sea necesario para cubrir nuestros objetivos de aprovechamiento, estos servidores pueden atender en un sólo proceso a miles o decena de miles de conexiones.

Si queremos realizar un programa que aproveche las diferentes CPUs y pueda realizar múltiples tareas a la vez tenemos muchos mecanismos para llevar esta tarea a cabo. Dependiendo del uso que se quiera dar probablemente queramos usar hilos o procesos, es aquí donde querremos escribir nuestro código con hilos o procesos.

**Hola Mundo**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
def cuadrado(x):
    return x**2

In [3]:
data = [i for i in range(10_000_000)]

In [4]:
%%time

seq = [cuadrado(x) for x in data]

seq[:5]

CPU times: user 2.53 s, sys: 262 ms, total: 2.79 s
Wall time: 3.02 s


[0, 1, 4, 9, 16]

In [5]:
%%time

map(cuadrado, data)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [6]:
%%time

seq=list(map(cuadrado, data))

seq[:5]

CPU times: user 2.3 s, sys: 303 ms, total: 2.61 s
Wall time: 2.87 s


[0, 1, 4, 9, 16]

In [7]:
import multiprocessing as mp

In [8]:
# movida del Mac M1

mp.get_start_method()

'spawn'

In [9]:
# movida del mac M1, para otros no hace falta

from multiprocessing import get_context

In [10]:
mp.cpu_count()    # nº de nucleos

8

In [11]:
%%time

#pool = mp.Pool(mp.cpu_count())    # usar todos los nucleos (Intel o AMD)

pool = get_context('fork').Pool(6)  # ARM M1  (en vez de ir de 1en1 va de 8en8)
 
seq = pool.map(cuadrado, data)

pool.close()

seq[:5]

CPU times: user 588 ms, sys: 456 ms, total: 1.04 s
Wall time: 1.38 s


[0, 1, 4, 9, 16]

**multiprocessing asíncrono**

`map` consume su iterable convirtiendo el iterable en una lista, dividiéndolo en fragmentos y enviando esos fragmentos a los procesos de trabajo en el Pool. Dividir el iterable en fragmentos funciona mejor que pasar cada elemento en el iterable entre procesos un elemento a la vez, especialmente si el iterable es grande. Sin embargo, convertir el iterable en una lista para dividirlo puede tener un costo de memoria muy alto, ya que la lista completa deberá mantenerse en la memoria.

`imap`/`map_async` no convierte el iterable que le da en una lista, ni lo divide en trozos. Itera sobre el elemento de uno en uno y los envia a un proceso de trabajo distinto. Esto significa que no se toma el golpe de memoria de convertir todo el iterable en una lista, pero también que el rendimiento es más lento para los iterables grandes, debido a la falta de fragmentación. Esto se puede mitigar aumentando el valor predeterminado de 1 en el `chunksize`. Otra gran diferencia de `imap` es que puede comenzar a recibir resultados de los trabajadores tan pronto como estén listos, en lugar de tener que esperar a que todos terminen. 




In [12]:
%%time

#pool=mp.Pool(mp.cpu_count())

pool=get_context('fork').Pool(6)  # grupo con 6 cores

res=pool.map_async(cuadrado, data).get()

pool.close()

res[:5]

CPU times: user 551 ms, sys: 418 ms, total: 969 ms
Wall time: 1.38 s


[0, 1, 4, 9, 16]

```python
%%time
pool=mp.Pool(mp.cpu_count())   

for x in pool.imap(cuadrado, datos):
    print(x)
    
pool.close()
```

$$$$

$$$$

## Joblib

![joblib](images/joblib.svg)

$$$$

$$$$


[Joblib](https://joblib.readthedocs.io/en/latest/) es una librería de Python que también nos permite paralelizar un programa. En este caso a través de procesos, lo cuál implica, como vimos antes, cierto tiempo para construir el Pool. Lo usaremos principalmente para realizar un bucle sobre una función.

Veamos el Hola Mundo.

**Hola Mundo**

In [ ]:
%pip install joblib

In [13]:
from joblib import Parallel, delayed

In [14]:
%%time

paralelo = Parallel(n_jobs=-1,   # n_jobs=-1 significa todos los cores
                    verbose=True
                   )


seq = paralelo(delayed(cuadrado)(e) for e in data)


seq[:5]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 16392 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 1785864 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 4653064 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 8339464 tasks      | elapsed:   37.3s


CPU times: user 32.8 s, sys: 5.92 s, total: 38.7 s
Wall time: 44.6 s


[Parallel(n_jobs=-1)]: Done 10000000 out of 10000000 | elapsed:   44.5s finished


[0, 1, 4, 9, 16]

### Ejemplo ESPN

Volvamos de nuevo al ejemplo de scrapeo de la págine de ESPN. Usaremos joblib para realizar una extracción en paralelo de la información.

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time

import pandas as pd

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH='driver/chromedriver'

In [16]:
url='https://www.espn.com/soccer/competitions'


driver=webdriver.Chrome(PATH)
driver.get(url)
    
time.sleep(2)

aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
aceptar.click()

time.sleep(4)

equipos=driver.find_element(By.CSS_SELECTOR, '#fittPageContainer > div.page-container.cf > div > div.layout__column.layout__column--1 > div > div:nth-child(3) > div:nth-child(1) > div > div:nth-child(5) > div > section > div > div > span:nth-child(2) > a')
equipos.click()


time.sleep(2)

equipos_stats_urls=driver.find_elements(By.CSS_SELECTOR, 'a.AnchorLink')

equipos_stats_urls=[e.get_attribute('href') for e in equipos_stats_urls 
                    if 'team/stats' in e.get_attribute('href')]


equipos_stats_urls[:20]

['https://www.espn.com/soccer/team/stats/_/id/3802/afc-wimbledon',
 'https://www.espn.com/soccer/team/stats/_/id/2731/accrington-stanley',
 'https://www.espn.com/soccer/team/stats/_/id/397/barnsley',
 'https://www.espn.com/soccer/team/stats/_/id/642/barrow',
 'https://www.espn.com/soccer/team/stats/_/id/392/birmingham-city',
 'https://www.espn.com/soccer/team/stats/_/id/365/blackburn-rovers',
 'https://www.espn.com/soccer/team/stats/_/id/346/blackpool',
 'https://www.espn.com/soccer/team/stats/_/id/358/bolton-wanderers',
 'https://www.espn.com/soccer/team/stats/_/id/387/bradford-city',
 'https://www.espn.com/soccer/team/stats/_/id/333/bristol-city',
 'https://www.espn.com/soccer/team/stats/_/id/308/bristol-rovers',
 'https://www.espn.com/soccer/team/stats/_/id/2567/burton-albion',
 'https://www.espn.com/soccer/team/stats/_/id/351/cambridge-united',
 'https://www.espn.com/soccer/team/stats/_/id/347/cardiff-city',
 'https://www.espn.com/soccer/team/stats/_/id/322/carlisle-united',
 'http

In [17]:
driver.quit()

In [18]:
def extraer(url):

    # inicia el driver
    driver=webdriver.Chrome(PATH)
    driver.get(url)

    time.sleep(2)

    # acepta cookies
    aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    aceptar.click()

    time.sleep(2)
    
    data=[]
    cabeceras=[]
    
    try:
        # dropdown
        dropdown = driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div/section/div/div[4]/select[1]')
        select = Select(dropdown)
        select.select_by_visible_text('2022-23')


        time.sleep(1)

        # disciplina
        dis=driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
        dis.click()

        time.sleep(2)

        tabla=driver.find_element(By.TAG_NAME, 'tbody')

        filas=tabla.find_elements(By.TAG_NAME, 'tr')

        for f in filas:

            elementos=f.find_elements(By.TAG_NAME, 'td') 

            tmp=[]

            for e in elementos:

                tmp.append(e.text)

            tmp.append(url.split('/')[-1])  # nombre del equipo
            data.append(tmp)


        cabeceras=driver.find_element(By.TAG_NAME, 'thead')

        cabeceras=[c.text for c in cabeceras.find_elements(By.TAG_NAME, 'th')]+['TEAM']

    except:
        time.sleep(0.1)
        
    driver.quit()
    
    return pd.DataFrame(data, columns=cabeceras)

In [19]:
paralelo = Parallel(n_jobs=6,  verbose=True)


lst_df = paralelo(delayed(extraer)(url) for url in equipos_stats_urls[:20])

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_18931/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_18931/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_18931/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_18931/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_18931/1636708756.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/95/ms6dwls51ls1jq0t456d3r200000gn/T/ipykernel_18931/16367

In [20]:
len(lst_df)

20

In [21]:
lst_df[0].head()

,RK,NAME,P,YC,RC,PTS,TEAM
0,1,Harry Pell,28,8,0,8,afc-wimbledon
1,,Paul Kalambayi,26,8,0,8,afc-wimbledon
2,3,Josh Davison,37,7,0,7,afc-wimbledon
3,,Alex Woodyard,34,7,0,7,afc-wimbledon
4,,Will Nightingale,22,7,0,7,afc-wimbledon


In [22]:
df = pd.concat(lst_df)

In [23]:
df.shape

(334, 7)

In [24]:
df.TEAM.unique()

array(['afc-wimbledon', 'barnsley', 'barrow', 'birmingham-city',
       'blackburn-rovers', 'bolton-wanderers', 'bradford-city',
       'bristol-city', 'bristol-rovers', 'burton-albion',
       'cambridge-united', 'cardiff-city', 'charlton-athletic',
       'cheltenham-town', 'colchester-united', 'coventry-city',
       'crawley-town'], dtype=object)

In [25]:
df.NAME.unique()

array(['Harry Pell', 'Paul Kalambayi', 'Josh Davison', 'Alex Woodyard',
       'Will Nightingale', 'Lee Brown', 'James Tilley', 'Ethan Chislett',
       'Isaac Ogundere', 'Aaron Pierre', 'Jack Currie',
       'Ali Ibrahim Al-Hamadi', 'Huseyin Biler', 'Alex Pearce',
       'Nik Tzanev', 'Zach Robinson', 'Harry Griffiths', 'Aron Sasu',
       'Quaine Bartley', 'Alfie Bendle', 'Morgan Williams',
       'Nathan Broome', 'Mads Andersen', 'Liam Kitching', 'Luca Connell',
       'James Norwood', 'Herbie Kane', 'Adam Phillips', 'Nicky Cadden',
       'Jasper Moon', 'Devante Cole', 'Josh Benson', 'Jordan Williams',
       'Matthew Wolfe', 'Jonathan Russell', 'Callum Styles',
       'Robbie Cundy', 'Bradley Collins', 'Conor McCarthy',
       'Barry Cotter', 'Jamie Searle', 'Sam Foley', 'Josh Kay',
       'George Ray', 'Paul Farman', 'Robbie Gotts', 'Tyrell Warren',
       'Tom White', 'Mazeed Ogungbo', 'Rory Feely', 'Niall Canavan',
       'Gerard Garner', 'Ben Whitfield', 'Elliot Newby', 'Josh 

**Tip:** https://pypi.org/project/tqdm/

**Para barras de progreso**

In [ ]:
%pip install tqdm